In [2]:
import pandas as pd
import json
import os
import requests
from datetime import datetime, time

In [3]:

file1 = '../rawdata/불법주정차 신고현황(21년9월29일_22년10월31일).csv'
file2 = '../rawdata/불법주정차 신고현황(22.11월~23.10월).csv'
file3 = '../rawdata/불법주정차 신고현황(23년11월1일_24년3월13일).csv'

df_reports_1 = pd.read_csv(file1, encoding='utf-8')
df_reports_2 = pd.read_csv(file2, encoding='utf-8')
df_reports_3 = pd.read_csv(file3, encoding='utf-8')


In [4]:
display(df_reports_1.head())

,민원접수일,민원접수시간,주소,경도,위도
0,20210929,00:05:00,서울 강서구 화곡로66길 130,126.859911,37.555537
1,20210929,00:38:00,중구 다산로48길 33 단우물어린이집앞,127.018013,37.567483
2,20210929,00:38:00,서울특별시 중구 마장로 19,127.012793,37.569183
3,20210929,00:50:00,서울특별시 강서구 까치산로15길 21,126.847125,37.547040
4,20210929,01:15:00,서울특별시 중구 다산로 293,127.015703,37.569085


In [5]:
print(df_reports_1.shape)
print(df_reports_2.shape)
print(df_reports_3.shape)

(1110253, 5)
(1466617, 5)
(493658, 5)


In [ ]:
#Add reset_index function
df_reports = pd.concat([df_reports_3, df_reports_1, df_reports_2], axis=0).reset_index(drop=True)
df_reports.shape

(3070528, 5)

In [8]:
file4 = '../rawdata/서울시 공영주차장 안내 정보.csv'
df_park_info = pd.read_csv(file4, encoding='cp949')

In [8]:
display(df_park_info.columns)

Index(['주차장코드', '주차장명', '주소', '주차장 종류', '주차장 종류명', '운영구분', '운영구분명', '전화번호',
       '주차현황 정보 제공여부', '주차현황 정보 제공여부명', '총 주차면', '유무료구분', '유무료구분명', '야간무료개방여부',
       '야간무료개방여부명', '평일 운영 시작시각(HHMM)', '평일 운영 종료시각(HHMM)', '주말 운영 시작시각(HHMM)',
       '주말 운영 종료시각(HHMM)', '공휴일 운영 시작시각(HHMM)', '공휴일 운영 종료시각(HHMM)',
       '최종데이터 동기화 시간', '토요일 유,무료 구분', '토요일 유,무료 구분명', '공휴일 유,무료 구분',
       '공휴일 유,무료 구분명', '월 정기권 금액', '노상 주차장 관리그룹번호', '기본 주차 요금',
       '기본 주차 시간(분 단위)', '추가 단위 요금', '추가 단위 시간(분 단위)', '버스 기본 주차 요금',
       '버스 기본 주차 시간(분 단위)', '버스 추가 단위 시간(분 단위)', '버스 추가 단위 요금', '일 최대 요금',
       '위도', '경도'],
      dtype='object')

In [11]:
df_park_info.head()

,주차장코드,주차장명,주소,주차장 종류,주차장 종류명,운영구분,운영구분명,전화번호,주차현황 정보 제공여부,주차현황 정보 제공여부명,...,기본 주차 시간(분 단위),추가 단위 요금,추가 단위 시간(분 단위),버스 기본 주차 요금,버스 기본 주차 시간(분 단위),버스 추가 단위 시간(분 단위),버스 추가 단위 요금,일 최대 요금,위도,경도
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,NW,노외 주차장,1,시간제 주차장,NaN,0,미연계중,...,NaN,300.0,10.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,NW,노외 주차장,1,시간제 주차장,02-2289-6735,0,미연계중,...,5.0,150.0,5.0,0.0,0.0,0.0,0.0,11000.0,NaN,NaN
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2,NW,노외 주차장,1,시간제 주차장,02-2204-7970,2,20~120분이내 연계데이터 존재(정보수집중),...,5.0,50.0,5.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,1025695,영등포여고 공영(구),영등포구 신길동 184-3,NW,노외 주차장,1,시간제 주차장,02-2677-1401,0,미연계중,...,5.0,50.0,5.0,0.0,0.0,0.0,0.0,0.0,37.515205,126.915116
4,1025696,당산근린공원 공영(구),영등포구 당산동3가 385-0,NW,노외 주차장,1,시간제 주차장,02-2677-1401,1,현재~20분이내 연계데이터 존재(현재 주차대수 표현),...,5.0,150.0,5.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:

# Rename columns
df_park_info = df_park_info.rename(columns={
    '총 주차면': '총주차면',
    '유무료구분': '평일유료',
    '평일 운영 시작시각(HHMM)': '평일시작',
    '평일 운영 종료시각(HHMM)': '평일종료',
    '주말 운영 시작시각(HHMM)': '토요일시작',
    '주말 운영 종료시각(HHMM)': '토요일종료',
    '공휴일 운영 시작시각(HHMM)': '공휴일시작',
    '공휴일 운영 종료시각(HHMM)': '공휴일종료',
    '토요일 유,무료 구분': '토요일유료',
    '공휴일 유,무료 구분': '공휴일유료',
    '기본 주차 요금': '기본주차요금',
    '기본 주차 시간(분 단위)': '기본주차시간',
    '추가 단위 요금': '추가단위요금',
    '추가 단위 시간(분 단위)': '추가단위시간'
})

# Define the desired column order
new_columns = ['주소', '총주차면', '평일유료', '토요일유료', '공휴일유료', '평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료', '운영구분', '기본주차요금', '기본주차시간', '추가단위요금', '추가단위시간', '경도', '위도']

# Reorder the columns in the DataFrame
df_parks = df_park_info[new_columns]

display(df_parks.head())
# display(df_parks.tail())


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
0,도봉구 창동 24-0,71.0,Y,N,N,900,1900,900,1900,900,1900,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,0,2400,0,2400,0,2400,126.915116,37.515205
4,영등포구 당산동3가 385-0,190.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN


In [17]:
def parse_time(value):

    if pd.isna(value):
        return None

    time_str = str(int(value)).zfill(4) if isinstance(value, (int, float)) else str(value)

    # Special-case “2400” or “24:00[:00]” as midnight
    if time_str in ("2400", "24:00", "24:00:00"):
        return time(0, 0)

    try:
        # Try parsing as HHMM
        return pd.to_datetime(time_str, format="%H%M").time()

    except:
        return None

In [18]:
time_columns = ['평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료']
for col in time_columns:
    df_parks[col] = df_parks[col].apply(parse_time).copy()

display(df_parks.head())

/var/folders/pf/y0k140gn11d95ybm0dgtk6m80000gn/T/ipykernel_26347/50185423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parks[col] = df_parks[col].apply(parse_time).copy()


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.915116,37.515205
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [22]:
# Find rows with missing values in columns except '위도' or '경도'
missing_rows = df_parks[df_parks.drop(columns=['경도', '위도']).isnull().any(axis=1)]

# Display rows with missing values
print("Rows with missing values")
display(missing_rows)

# Drop rows with missing values in columns except '위도' or '경도'
df_parks_dropped = df_parks.drop(missing_rows.index)


Rows with missing values


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
7688,종로구 명륜4가 105-3,NaN,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8058,용산구 남영동 71-2,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8059,용산구 남영동 71-2,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8073,송파구 가락동 156-11,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8080,송파구 문정동 640-9,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8278,송파구 풍납동 306-25,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8279,송파구 풍납동 222-75,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8280,송파구 풍납동 292-33,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8281,송파구 풍납동 140-3,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [23]:
df_parks_dropped = df_parks_dropped.reset_index(drop=True)
# Display the updated DataFrame
display(df_parks_dropped)


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.915116,37.515205
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8327,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8328,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,NaN,NaN
8329,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [25]:
# Find rows in df_parks_dropped with missing coordinates
df_parks_missing = df_parks_dropped[df_parks_dropped.isnull().any(axis=1)]

# Drop these rows from df_parks_dropped
df_parks_dropped = df_parks_dropped.drop(df_parks_missing.index)

# Display the DataFrame with missing rows and the cleaned DataFrame
display(df_parks_missing)
display(df_parks_dropped)

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
5,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8327,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8328,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,NaN,NaN
8329,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.915116,37.515205
6,동대문구 휘경동 308-32,78.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,127.058093,37.588069
10,강남구 압구정동 428-0,344.0,Y,N,N,09:00:00,21:00:00,09:00:00,21:00:00,09:00:00,21:00:00,127.026471,37.527017
12,강남구 개포동 1273-0,194.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.052365,37.475646
16,구로구 구로동 810-3,91.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.901243,37.485432
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8305,강북구 수유동 535-76,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,09:00:00,18:00:00,127.002927,37.643145
8308,강서구 화곡동 809-26,203.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.861058,37.534312
8310,강서구 화곡동 890-0,248.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.848861,37.529046
8318,양천구 목동 905-32,48.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,126.876610,37.533440


In [26]:
# Function to get coordinates for a given address
def get_coordinates(address):
    apiurl = "https://api.vworld.kr/req/address?"
    apikey =  "D9C1CFC9-0FD3-3304-A3E7-131C33A91190"

    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",
        "key": apikey
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            point_info = data['response']['result']['point']
            return point_info['x'], point_info['y']
    return None, None


In [27]:
# Geocode missing coordinates
df_parks_missing[['경도', '위도']] = df_parks_missing['주소'].apply(lambda addr: pd.Series(get_coordinates(addr)))

# Display the updated dataframe
df_parks_missing

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.03510376366461,37.56996805245046
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89577762856109,37.52554507178421
5,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89444474098582,37.49923824170827
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.01847091587678,37.622737640361635
8327,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.02067861196652,37.63323147926457
8328,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,126.9565346583981,37.612297739999235
8329,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89159944589176,37.50731316416754


In [31]:
df_parks_combined = pd.concat([df_parks_missing, df_parks_dropped], ignore_index=True)

# Display the combined DataFrame
display(df_parks_combined)

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.03510376366461,37.56996805245046
3,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89577762856109,37.52554507178421
4,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89444474098582,37.49923824170827
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 수유동 535-76,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,09:00:00,18:00:00,127.002927,37.643145
8327,강서구 화곡동 809-26,203.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.861058,37.534312
8328,강서구 화곡동 890-0,248.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.848861,37.529046
8329,양천구 목동 905-32,48.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,126.87661,37.53344


In [32]:
# Display rows with missing values
missing_rows = df_parks_combined[df_parks_combined.isnull().any(axis=1)]
display(missing_rows.head())

# Drop rows with coordinate still missing after geocoding
df_parks_cleaned = df_parks_combined.dropna()

# Display the cleaned DataFrame
display(df_parks_cleaned)

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
73,강북구 미아동 137-19,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,None,None
92,강북구 수유동 55-53,116.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,None,None
111,종로구 장사동 116-3,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,None,None
127,종로구 창신동 37-16,1.0,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,None,None
128,종로구 인사동 283-0,1.0,Y,N,Y,10:00:00,22:00:00,09:00:00,17:00:00,00:00:00,00:00:00,None,None


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.03510376366461,37.56996805245046
3,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89577762856109,37.52554507178421
4,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89444474098582,37.49923824170827
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 수유동 535-76,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,09:00:00,18:00:00,127.002927,37.643145
8327,강서구 화곡동 809-26,203.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.861058,37.534312
8328,강서구 화곡동 890-0,248.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.848861,37.529046
8329,양천구 목동 905-32,48.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,126.87661,37.53344


In [33]:
df_parks_cleaned.to_csv('parks.csv', index=False, encoding='utf-8-sig')

In [34]:
df_reports

,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
1466612,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
1466613,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
1466614,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
1466615,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [35]:
# Find rows with missing values in columns except '위도' or '경도'
missing_rows = df_reports[df_reports.drop(columns=['경도', '위도']).isnull().any(axis=1)]

# Display rows with missing values
display(missing_rows)

# Drop rows with missing values in columns except '위도' or '경도'
df_reports_dropped = df_reports.drop(missing_rows.index)

# Display the updated DataFrame
display(df_reports_dropped)

,민원접수일,민원접수시간,주소,경도,위도
2727,20231101,18:49:00,NaN,127.067222,37.664983
3092,20231101,19:51:00,NaN,127.067170,37.664344
3136,20231101,20:02:00,NaN,126.906070,37.442495
20860,20231106,07:34:00,NaN,127.059652,37.669789
27235,20231107,22:04:00,NaN,127.071881,37.645576
...,...,...,...,...,...
1465939,20231031,20:04:00,NaN,0.000000,0.000000
1465987,20231031,20:17:00,NaN,0.000000,0.000000
1466103,20231031,20:45:00,NaN,127.063964,37.645916
1466558,20231031,23:22:00,NaN,0.000000,0.000000


,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
1466612,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
1466613,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
1466614,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
1466615,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [36]:
# Find rows in df_reports_dropped with any missing values
df_reports_missing = df_reports_dropped[df_reports_dropped.isnull().any(axis=1)]

# Drop these rows from df_reports_dropped
df_reports_dropped = df_reports_dropped.drop(df_reports_missing.index)

# Display the DataFrame with missing rows and the cleaned DataFrame
print('missing: ', df_reports_missing.shape)
display(df_reports_dropped)

missing:  (27, 5)


,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
1466612,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
1466613,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
1466614,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
1466615,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [37]:
missing_values = df_reports_dropped.isnull().sum()
print(missing_values)

민원접수일     0
민원접수시간    0
주소        0
경도        0
위도        0
dtype: int64


In [38]:
df_reports_cleaned = df_reports_dropped.copy()
df_reports_cleaned.head()

,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286


In [39]:
pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [40]:
import holidays

df_reports_cleaned['민원접수일'] = pd.to_datetime(df_reports_cleaned['민원접수일'].astype(str), format='%Y%m%d')

# Load Korean public holidays for all relevant years
years = df_reports_cleaned['민원접수일'].dt.year.unique().tolist()
kr_holidays = holidays.KR(years=years)


In [41]:
# Define a function to classify each date
def classify_day(d):
    wd = d.weekday()  # Mon=0 … Sat=5, Sun=6
    if wd == 5:
        return 'Saturday'
    # treat Sunday or any official holiday as Holiday
    elif wd == 6 or d in kr_holidays:
        return 'Holiday'
    else:
        return 'Weekday'


In [42]:

df_reports_cleaned['요일'] = df_reports_cleaned['민원접수일'].apply(classify_day)
df_reports_cleaned['요일'].value_counts()

요일
Weekday     2021957
Holiday      545670
Saturday     489410
Name: count, dtype: int64

In [46]:
# sort by 민원접수일
df_reports_cleaned.sort_values(by='민원접수일', ascending=True, inplace=True)
# Reset the index
df_reports_cleaned.reset_index(drop=True, inplace=True)
df_reports_cleaned

,민원접수일,민원접수시간,주소,경도,위도,요일
0,2021-09-29,19:29:00,서울특별시 강서구 강서로15길 49,126.843247,37.532089,Weekday
1,2021-09-29,00:05:00,서울 강서구 화곡로66길 130,126.859911,37.555537,Weekday
2,2021-09-29,00:38:00,중구 다산로48길 33 단우물어린이집앞,127.018013,37.567483,Weekday
3,2021-09-29,00:38:00,서울특별시 중구 마장로 19,127.012793,37.569183,Weekday
4,2021-09-29,00:50:00,서울특별시 강서구 까치산로15길 21,126.847125,37.547040,Weekday
...,...,...,...,...,...,...
3057032,2024-03-13,23:56:00,서울특별시 마포구 희우정로20길 71,126.904288,37.556222,Weekday
3057033,2024-03-13,23:48:00,(청룡동)남부순환로208길 10,126.943266,37.481659,Weekday
3057034,2024-03-13,22:44:00,우/강서로 312-15,126.837496,37.553985,Weekday
3057035,2024-03-13,23:49:00,서울특별시 은평구 연서로38길 26,126.929367,37.622501,Weekday


In [47]:
df_reports_cleaned.to_csv('reports.csv', index=False, encoding='utf-8-sig')